In [7]:
import torch
import numpy as np
from nerfstudio.data.pixel_samplers import PatchPixelSampler,PatchPixelSamplerConfig

In [12]:
sampler = PatchPixelSampler(
    PatchPixelSamplerConfig(patch_size=16)
)

In [75]:
def sample_method(
        patch_size,
        batch_size,
        image_height,
        image_width,
        mask= None,
        device= "cpu",
    ):
    if isinstance(mask, torch.Tensor):
        sub_bs = batch_size // (patch_size[0]*patch_size[1])
        half_patch_size = [int(patch_size[0] / 2),int(patch_size[1] / 2)]
        m = erode_mask(mask.permute(0, 3, 1, 2).float(), pixel_radius=half_patch_size)
        nonzero_indices = torch.nonzero(m[:, 0], as_tuple=False).to(device)
        chosen_indices = random.sample(range(len(nonzero_indices)), k=sub_bs)
        indices = nonzero_indices[chosen_indices]
    
        indices = (
            indices.view(sub_bs, 1, 1, 2)
            .broadcast_to(sub_bs, patch_size[0], patch_size[1], 2)
            .clone()
        )
    
        yys, xxs = torch.meshgrid(
            torch.arange(patch_size[0], device=device), torch.arange(patch_size[1], device=device)
        )
        indices[:, ..., 0] += yys - half_patch_size[0]
        indices[:, ..., 1] += xxs - half_patch_size[1]
    
        indices = torch.floor(indices).long()
        indices = indices.flatten(0, 2)
    else:
        half_patch_size = torch.tensor([int(patch_size[0] / 2),int(patch_size[1] / 2)],device=device)
        sub_bs = batch_size // (patch_size[0]*patch_size[1])
        indices = torch.rand((sub_bs, 2), device=device) * torch.tensor(
            [image_height , image_width],
            device=device,
        )+half_patch_size[None]
        indices = torch.clamp(indices,max=torch.tensor([image_height,image_width])) 
        indices = indices - torch.tensor([patch_size[0],patch_size[1]], device=device)[None]
        indices = torch.clamp(indices,min=torch.tensor([0,0])) 
        indices = (
            indices.view(sub_bs, 1, 1, 2)
            .broadcast_to(sub_bs, patch_size[0], patch_size[1], 2)
            .clone()
        )
    
        yys, xxs = torch.meshgrid(
            torch.arange(patch_size[0], device=device), torch.arange(patch_size[1], device=device)
        )
        indices[:, ..., 0] += yys
        indices[:, ..., 1] += xxs
    
        indices = torch.floor(indices).long()
        indices = indices.flatten(0, 2)
    
    return indices

In [83]:
 (patch_size[0]*patch_size[1])

4

In [87]:
patch_size = (4,4)
image_height =4 
image_width = 4
batch_size = 4*4
ind = sample_method(
        patch_size,
        batch_size,
        image_height,
        image_width)
ind = ind.reshape((ind.shape[0]//patch_size[0]//patch_size[1],patch_size[0],patch_size[1],2))

In [88]:
ind

tensor([[[[0, 0],
          [0, 1],
          [0, 2],
          [0, 3]],

         [[1, 0],
          [1, 1],
          [1, 2],
          [1, 3]],

         [[2, 0],
          [2, 1],
          [2, 2],
          [2, 3]],

         [[3, 0],
          [3, 1],
          [3, 2],
          [3, 3]]]])

In [73]:
ind[0].flatten(0,1)

tensor([[  0, -24],
        [  0, -23],
        [  0, -22],
        ...,
        [239, 293],
        [239, 294],
        [239, 295]])

In [ ]:
ind = sample_method(
        patch_size = 32,
        batch_size = 240*320,
        image_height = 240,
        image_width = 320)

In [36]:
data = torch.tensor([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]])
data

tensor([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12],
        [13, 14, 15, 16]])

In [22]:
h =4
w =4
pw = 2
data.reshape((1,h//pw,pw,w//pw,pw)).permute(0,1,3,2,4).reshape((1,-1,pw,pw))

tensor([[[[ 1,  2],
          [ 5,  6]],

         [[ 3,  4],
          [ 7,  8]],

         [[ 9, 10],
          [13, 14]],

         [[11, 12],
          [15, 16]]]])

In [62]:
torch.clamp(torch.tensor([[100,100]]),max=torch.tensor([50,100]))

tensor([[ 50, 100]])